<center><img src="img/skillfactorylogo.png"></center>

<h1><center>Курс "Практический Machine Learning"</center></h1>
<h3><center>Шестаков Андрей</center></h3>
<hr>
<h2><center>Введение в рекомендательные системы</center></h2>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 8)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from ipywidgets import interact, IntSlider, fixed, FloatSlider

# Мотивация

* Люди - потребители контента и услуг
    * Музыка
    * Фильмы
    * Книги
    * Игры
    * Еда
    * ...


* Но выбор слишком велик..
    * Spotify - 30 млн. песен
    * Netflix - 20 тыс. фильмов
    * Amazon - 500 тыс. книг
    * Steam - 20 тыс. игр

Надо как-то фильтровать..
* Можно спросить у друзей (вкусы могут отличаться)
* Можно почитать обзоры (много времени)
* Автоматическая рекомендательная система!

<center><img src="img/recsys.jpg" width=400></center>



### Netflix Prize
<center><img src="img/netflix.jpg" width=500></center>

### Источники персональных рекомендаций
* На основе предпочтений пользователя
    * Рассчитывается некоторый "профиль" пользователя, для которого определяются наиболее подходящие товары
* На основе похожих пользователей
    * Находим других пользователей с похожими интересами и доставляем рекомендацию
* На основе похожих товоров
    * Рекомендуем товары, похожие на те, что мне нравятся

# Постановка проблемы

* Пользователи ставят оценку товарам
    * Бинарную
    * Количество "звезд"
    * Неявную (кол-во потраченого времени\денег)
<center><img src="img/rating.png" width=600></center>
* Надо заполнить пропуски
* Предоставить рекомендацию



### Нюансы
* Хорошее восстановление рейтингов $\neq$ хорошая рекомендательная система
* Учет экономических предпочтений продавцов
* Learning Loop
* Холодный старт
* Возникает для новых товаров и пользователей
* Масштабируемость
* Накручивание рейтингов
* Неактивные пользователи
* Тривиальные рекомендации

<center><img src="img/hp-rec.png" width=600></center>

## Подходы к решению
* Коллаборативная фильтрация
* Латентные методы (матричные разложения)

# Коллаборативная фильтрация

* User-based
* Item-based

## User-based CF
<center><img src="img/ub-collab.png" width=700></center>

Введем обозначения:
* $U$ - множество пользователей
* $I$ - множество товаров
* $U_i$ - множество пользователей, оценивших товар $i$
* $I_u$ - множество товаров, оценненных пользователем $u$
* $R_{ui}$ - оценка, которую дал пользователь $u$ товару $i$
* $\hat{R}_{ui}$ - прогноз оценки



### Прогнозирование рейтинга
* Посчитаем сходство между пользователями $s \in \mathbb{R}^{U \times U}$
* Для целевого пользователя $u$ найти похожих пользователей $N(u)$
$$ \hat{R}_{ui} = \bar{R}_u + \frac{\sum_{v \in N(u)} s_{uv}(R_{vi} - \bar{R}_v)}{\sum_{v \in N(u)} \left| s_{uv}\right|} $$

* $\bar{R}_u$ - поправка на писсимизм\оптимизм пользователей

### Как определять $N(u)$?
* $N(u)$ можно определять по разным соображениям:
    * Брать всех
    * Top-k
    * $s_{uv} > \theta$

### Как определять схожесть пользователей

* Для каждой пары $(u,v)$ надо пересечь множество оцененных товаров

* Корреляция пирсона
$$ s_{uv} = \frac{\sum\limits_{i \in I_u\cap I_v} (R_{ui} - \bar{R}_u)(R_{vi} - \bar{R}_v)}{\sqrt{\sum\limits_{i \in I_u\cap I_v}(R_{ui} - \bar{R}_u)^2}\sqrt{\sum\limits_{i \in I_u\cap I_v}(R_{vi} - \bar{R}_v)^2}}$$
* Корреляция Спирмана
* Косинусная мера
$$ s_{uv} = \frac{\sum\limits_{i \in I_u\cap I_v} R_{ui} R_{vi}}{\sqrt{{\sum\limits_{i \in I_u\cap I_v}R_{ui}^2}}\sqrt{{\sum\limits_{i \in I_u\cap I_v}R_{vi}^2}}}$$


### Пример

<img src='img/example1.png'>

<img src='img/example2.png'>

## Item-based CF

<center><img src='http://dataconomy.com/wp-content/uploads/2015/03/Beginners-Guide-Recommender-Systems-Content-Based-Filtering.png' width=350></center>

### Прогнозирование рейтинга
* Посчитаем сходство между товарами $s \in \mathbb{R}^{I \times I}$
* Для товара $i$ найти оцененные пользователем $u$ похожие товары: $N(i)$

$$ \hat{R}_{ui} = \frac{\sum_{j \in N(i)} s_{ij}R_{uj}}{\sum_{j \in N(i)} \left| s_{ij}\right|} $$

### Схожесть товаров

* Условная вероятность
$$ s_{ij} = \frac{n_{ij}}{n_i} $$
* Зависимость
$$ s_{ij} = \frac{n_{ij}}{n_i n_j} $$

Попробуем что-то сделать с модулем [surprice](http://surprise.readthedocs.io/en/stable/index.html)

## Демо CF

In [ ]:
filepath = './data/user_ratedmovies.dat'
df_rates = pd.read_csv(filepath, sep='\t')

filepath = './data/movies.dat'
df_movies = pd.read_csv(filepath, sep='\t', encoding='iso-8859-1')

In [ ]:
df_movies.head()

In [ ]:
df_movies.loc[:, 'id'] = df_movies.loc[:, 'id'].astype('str')
df_movies = df_movies.set_index('id')

In [ ]:
df_rates.head()

In [ ]:
q = df_rates.datetime.quantile(0.85)

In [ ]:
filepath = './data/user_ratedmovies_train.dat'
idx = df_rates.datetime < q
df_rates.loc[idx].to_csv(filepath, sep='\t', columns=['userID', 'movieID', 'rating'], index=None)

filepath = './data/user_ratedmovies_test.dat'
df_rates.loc[~idx].to_csv(filepath, sep='\t', columns=['userID', 'movieID', 'rating'], index=None)

In [ ]:
from surprise import Dataset

In [ ]:
filepaths = [('./data/user_ratedmovies_train.dat', './data/user_ratedmovies_test.dat')]
reader = Reader(line_format='user item rating', sep='\t', skip_lines=1)
data = Dataset.load_from_folds(filepaths, reader=reader)

In [ ]:
from surprise import KNNBasic, KNNWithMeans
from surprise.accuracy import rmse
from surprise import dump

Описание алгоритмов, основанных на CF - [туть](http://surprise.readthedocs.io/en/stable/knn_inspired.html)

In [ ]:
sim_options = {'name': 'cosine',
               'user_based': True
               }

In [ ]:
dumpfile = './alg.dump'

In [ ]:
dump.dump(dumpfile, predictions, algo)

In [ ]:
algo = KNNWithMeans(k=20, min_k=1, sim_options=sim_options)                                                       

for trainset, testset in data.folds(): 
    algo.train(trainset)                             
    predictions = algo.test(testset)
    rmse(predictions)
    
    dump.dump(dumpfile, predictions, algo)

In [ ]:
df_predictions = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details']) 

In [ ]:
df_predictions.head()

In [ ]:
algo.predict('190', '173', verbose=2)

In [ ]:
anti_train = trainset.build_anti_testset()

In [ ]:
one_user = filter(lambda r: r[0] == '75', anti_train)

In [ ]:
# Это будет долго..
# anti_train_predictions = algo.test(one_user)

anti_train_predictions = algo.test(one_user)

In [ ]:
from collections import defaultdict

def get_top_n(predictions, n=10):

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [ ]:
df_movies.loc['5695', 'title']

In [ ]:
top_n = get_top_n(anti_train_predictions, n=10)

for uid, user_ratings in top_n.items():
    print(uid, [df_movies.loc[iid, 'title'] for (iid, _) in user_ratings])

# Модели со скрытыми факторами

Для каждого пользователя и товара построим векторы $p_u\in \mathbb{R}^{k}$ и $q_i \in \mathbb{R}^{k}$ так, чтобы
$$ R_{ui} \approx p_u^\top q_i $$

* $p_u$ иногда получается интерпретировать как заинтересованность пользователя в некоторой категории товаров
* $q_i$ иногда получается интерпретировать как принадлежность товара к определенной категории

Кроме того, в полученном пространстве, можно считать похожесть пользователей и товаров

<center><img src='img/matrix_factorization.png' width=600></center>

## Non-negative Matrix Factorization

* $P \geq 0$
* $Q \geq 0$

## SVD разложение
<center><img src='img/svd.png' width=600></center>

* Надо чем-то заполнить пропуски
    * Нулями
    * Базовыми предсказаниями
* Как вариант 
    * $R' = R-B$ и заполнить $0$
* Таким образом:
    * $P = U\sqrt{\Sigma}$
    * $Q = \sqrt{\Sigma}V^\top$
    * $\hat{R} = P^\top Q$
* А как делать предсказания для новых пользователей?

## Non-negative Matrix Factorization

* $P \geq 0$
* $Q \geq 0$

##  Latent Factor Model
* Будем оптимизировать следующий функционал
$$ \sum\limits_{u,i}(R_{ui} - \bar{R}_u - \bar{R}_i - \langle p_u, q_i \rangle)^2 + \lambda \sum_u\| p_u \|^2 + \mu\sum_i\| q_i \|^2 \rightarrow \min\limits_{P, Q} $$
* С помощью градиентного спуска (на каждом шаге случайно выбирая пару $(u,i)$:
$$ p_{uk} = p_{uk} + 2\alpha \left(q_{ik}(R_{ui} - \bar{R}_u - \bar{R}_i - \langle p_u, q_i \rangle) - \lambda p_{uk}\right)$$
$$ q_{ik} = q_{ik} + 2\alpha \left(p_{uk}(R_{ui} - \bar{R}_u - \bar{R}_i - \langle p_u, q_i \rangle) - \mu q_{ik}\right)$$

## Демо SVD

### Перекодируем ID фильмов и пользователей

In [ ]:
filepath = './data/user_ratedmovies.dat'
df_rates = pd.read_csv(filepath, sep='\t')

filepath = './data/movies.dat'
df_movies = pd.read_csv(filepath, sep='\t', encoding='iso-8859-1')

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
mov_enc = LabelEncoder()
mov_enc.fit(df_rates.movieID.values)
n_movies = df_rates.movieID.nunique()

In [ ]:
user_enc = LabelEncoder()
user_enc.fit(df_rates.userID.values)
n_users = df_rates.userID.nunique()

In [ ]:
idx = df_movies.loc[:, 'id'].isin(df_rates.movieID)
df_movies = df_movies.loc[idx, :]

In [ ]:
df_rates.loc[:, 'movieID'] = mov_enc.transform(df_rates.movieID.values)
df_movies.loc[:, 'id'] = mov_enc.transform(df_movies.loc[:, 'id'].values)
df_rates.loc[:, 'userID'] = user_enc.transform(df_rates.userID.values)

In [ ]:
df_rates.head()

### В явном виде запишем матрицу рейтингов

In [ ]:
from scipy.sparse import coo_matrix, csr_matrix

In [ ]:
n_users_train = df_rates.userID.nunique()
R_train = coo_matrix((df_rates.rating, 
                     (df_rates.userID.values, df_rates.movieID.values)),
                     shape=(n_users, n_movies))

In [ ]:
from scipy.sparse.linalg import svds

In [ ]:
u, s, vt = svds(R_train, k=10, )

In [ ]:
vt.shape

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
nn = NearestNeighbors(n_neighbors=10, metric='cosine')

In [ ]:
v = vt.T

In [ ]:
nn.fit(v)

In [ ]:
ind = nn.kneighbors(v, return_distance=False)

In [ ]:
m_names = df_movies.title.values

In [ ]:
m_names = pd.DataFrame(data=m_names[ind], columns=['movie']+['nn_{}'.format(i) for i in range(1,10)])

In [ ]:
idx = m_names.movie.str.contains('Terminator')
m_names.loc[idx]

# Оценка качества

* Качество рейтингов
    * MAE, MSE
* Качество событий
    * F-score, ROC-AUC, PR-AUC
    * precision@k, recall@k